In [1]:
# prompt: connect to google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pkill -f app.py


In [3]:
!fuser -k 5000/tcp


In [4]:
!pip install cvzone==1.5.6
!pip install ultralytics
!pip install hydra-core
!pip install matplotlib
!pip install numpy
!pip install opencv-python
!pip install Pillow
!pip install PyYAML
!pip install requests
!pip install scipy
!pip install torch
!pip install torchvision
!pip install tqdm
!pip install filterpy
!pip install scikit-image
!pip install lap

  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.5.6-py3-none-any.whl size=18748 sha256=f9600baa21cd824044b523cbcf3db710889388559398a9b9e3a80ade37fd9dae
  Stored in directory: /root/.cache/pip/wheels/53/1c/bf/ca6e08f684e2097136bda44291fa2af2ce16a8049307aa7e05
Successfully built cvzone
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=bb0728bbfc93e1413a794402f20737733763f228ef1c8c880990435db2905939
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.0 MB/s eta 0:00:00


In [8]:
from ultralytics import YOLO
import cv2
import cvzone
import math
from google.colab.patches import cv2_imshow  # Import cv2_imshow for Colab

# Set up video input and output
input_path = "/content/drive/MyDrive/DataCrumbs/Object Detection/Object Detection/Videos/2.mp4"
output_path = "/content/drive/MyDrive/DataCrumbs/Object Detection/Object Detection/pre-videos/2.mp4"

cap = cv2.VideoCapture(input_path)  # Load input video
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get FPS of the input video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Video width
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Video height
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video codec
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))  # Create VideoWriter object

# Load YOLO model
model = YOLO("/content/drive/MyDrive/DataCrumbs/Object Detection/Object Detection/ppe.pt")

# Define class names and colors
classNames = ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone',
              'Safety Vest', 'machinery', 'vehicle']
myColor = (0, 0, 255)

while True:
    success, img = cap.read()
    if not success:
        break  # Exit the loop if the video ends or fails to load

    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]
            print(currentClass)

            if conf > 0.5:
                if currentClass == 'NO-Hardhat' or currentClass == 'NO-Safety Vest' or currentClass == "NO-Mask":
                    myColor = (0, 0, 255)
                elif currentClass == 'Hardhat' or currentClass == 'Safety Vest' or currentClass == "Mask":
                    myColor = (0, 255, 0)
                else:
                    myColor = (255, 0, 0)

                # Add text and bounding box to the frame
                cvzone.putTextRect(img, f'{classNames[cls]} {conf}',
                                   (max(0, x1), max(35, y1)), scale=1, thickness=1, colorB=myColor,
                                   colorT=(255, 255, 255), colorR=myColor, offset=5)
                cv2.rectangle(img, (x1, y1), (x2, y2), myColor, 3)

    # Save the processed frame to the output video
    out.write(img)

    # Display the frame in Colab
    # cv2_imshow(img)

cap.release()
out.release()
print(f"Processed video saved at: {output_path}")


Streaming output truncated to the last 5000 lines.
Safety Vest
Hardhat
NO-Mask
NO-Hardhat
Hardhat
NO-Mask
NO-Safety Vest
Person
Speed: 3.9ms preprocess, 40.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Hardhats, 1 Mask, 1 NO-Hardhat, 3 NO-Masks, 2 NO-Safety Vests, 5 Persons, 2 Safety Vests, 39.9ms
Safety Vest
Person
Person
NO-Safety Vest
Hardhat
Person
Hardhat
NO-Mask
Safety Vest
Hardhat
NO-Mask
Mask
NO-Hardhat
Hardhat
NO-Mask
Person
NO-Safety Vest
Person
Speed: 3.9ms preprocess, 39.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Hardhats, 2 Masks, 1 NO-Hardhat, 3 NO-Masks, 2 NO-Safety Vests, 5 Persons, 2 Safety Vests, 40.0ms
Person
Safety Vest
Person
NO-Safety Vest
Hardhat
Person
Hardhat
NO-Mask
Safety Vest
Hardhat
NO-Mask
NO-Hardhat
Person
Hardhat
NO-Safety Vest
NO-Mask
Mask
Person
Mask
Speed: 6.1ms preprocess, 40.0ms inference, 4.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Hardhats, 2 Masks

# Flask

In [16]:
%%writefile app.py
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from ultralytics import YOLO
import cv2
import cvzone
import math
import os

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Load YOLO model
MODEL_PATH = "/content/drive/MyDrive/DataCrumbs/Object Detection/Object Detection/ppe.pt"
model = YOLO(MODEL_PATH)

# Define class names and colors
classNames = ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone',
              'Safety Vest', 'machinery', 'vehicle']

@app.route("/detect_video", methods=["POST"])
def detect_video():
    try:
        if "file" not in request.files:
            return jsonify({"error": "No video uploaded"}), 400

        # Save uploaded video
        file = request.files["file"]
        input_path = "input_video.mp4"
        file.save(input_path)

        # Set output path
        output_path = "output_video.mp4"

        # Open video file
        cap = cv2.VideoCapture(input_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        if fps == 0 or width == 0 or height == 0:
            return jsonify({"error": "Invalid video file"}), 400

        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        while True:
            success, img = cap.read()
            if not success:
                break

            results = model(img, stream=True)
            for r in results:
                boxes = r.boxes
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                    conf = math.ceil((box.conf[0] * 100)) / 100
                    cls = int(box.cls[0])
                    currentClass = classNames[cls]

                    if conf > 0.5:
                        if currentClass in ['NO-Hardhat', 'NO-Safety Vest', 'NO-Mask']:
                            myColor = (0, 0, 255)
                        elif currentClass in ['Hardhat', 'Safety Vest', 'Mask']:
                            myColor = (0, 255, 0)
                        else:
                            myColor = (255, 0, 0)

                        cvzone.putTextRect(img, f'{classNames[cls]} {conf}',
                                           (max(0, x1), max(35, y1)), scale=1, thickness=1, colorB=myColor,
                                           colorT=(255, 255, 255), colorR=myColor, offset=5)
                        cv2.rectangle(img, (x1, y1), (x2, y2), myColor, 3)

            out.write(img)

        cap.release()
        out.release()

        return send_file(output_path, as_attachment=True, mimetype="video/mp4")

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)


Overwriting app.py


In [17]:
!pip install flask flask-cors pyngrok ultralytics opencv-python-headless

import threading
import time
from pyngrok import ngrok

# Start Flask in the background
def run_flask():
    !python3 app.py

thread = threading.Thread(target=run_flask)
thread.start()

time.sleep(3)  # Wait for Flask to start

# Expose Flask with Ngrok
public_url = ngrok.connect(5000).public_url
print(f"Flask app running at: {public_url}")


Flask app running at: https://ca68-34-124-158-25.ngrok-free.app


In [18]:
import requests

url = "https://ca68-34-124-158-25.ngrok-free.app/detect_video"  # Replace with actual Ngrok URL
video_path = "/content/drive/MyDrive/DataCrumbs/Object Detection/Object Detection/Videos/4.mp4"  # Update this

files = {"file": open(video_path, "rb")}
response = requests.post(url, files=files)

if response.status_code == 200:
    with open("downloaded_output.mp4", "wb") as f:
        f.write(response.content)
    print("Downloaded processed video as 'downloaded_output.mp4'")
else:
    print("Error:", response.json())



0: 384x640 6 Hardhats, 1 NO-Mask, 4 NO-Safety Vests, 5 Persons, 1 Safety Vest, 90.6ms
Speed: 4.3ms preprocess, 90.6ms inference, 242.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Hardhats, 1 NO-Hardhat, 1 NO-Mask, 4 NO-Safety Vests, 4 Persons, 2 Safety Vests, 39.1ms
Speed: 3.0ms preprocess, 39.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Hardhats, 1 NO-Hardhat, 1 NO-Mask, 4 NO-Safety Vests, 3 Persons, 1 Safety Vest, 39.4ms
Speed: 2.4ms preprocess, 39.4ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Hardhats, 1 NO-Mask, 2 NO-Safety Vests, 4 Persons, 1 Safety Vest, 39.0ms
Speed: 2.5ms preprocess, 39.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 Hardhats, 1 NO-Mask, 4 NO-Safety Vests, 4 Persons, 1 Safety Vest, 34.0ms
Speed: 2.2ms preprocess, 34.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Hardhats, 1 NO-Mask, 2 NO-Safety Ve